# Updates/Installation

In [ ]:
'''


#Need to update? Here ya go!!


#!py -m pip uninstall astropy
#!py -m pip install git+https://github.com/astropy/astropy
#!pip install emcee
!pip install corner
    



!py -m pip install git+https://github.com/radio-astro-tools/spectral-cube.git
!py -m pip install reproject
!py -m pip install git+https://github.com/radio-astro-tools/spectral-cube.git 
!py -m pip install pyspeckit
!py -m pip install regions
!py -m pip install astrodendro
!py -m pip  install wcsaxes 
!py -m pip  install ipympl
!py -m pip install dask
!py -m pip install radio_beam
!py -m pip install casa_formats_io
#try:
#    !pip install casa_formats_io --no-binary :all:
#except:
#    !pip install casa_formats_io --no-cache --no-binary :all:

!py -m pip  install spectral_cube 
!py -m pip  install typing 
!py -m pip install mypy
!py -m pip  install typing_extensions 



'''




# Imports

In [1]:
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################

#Every data reduction and analysis file will use these imports and functions.
#So i run this file at the beggining of every other file to import stuff.

######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################

#These will show you what version of Python you are working with. Important because astropy works best with certain versions like 3.8.5

import sys, traceback

print(sys.executable)
print(sys.version)
print(sys.version_info)



import math
import os
import copy

# The most important package for astronomy

import astropy
from astropy.coordinates import SkyCoord
print('astropy',astropy.__version__ )
import astropy.io.fits as fits              
from astropy.wcs import WCS # World coordinate system
from astropy import units as u  
from astropy.table import Table
from astropy.convolution import Gaussian1DKernel
from astropy.utils import NumpyRNGContext


# Spectral cubes are amazing at taking fits images and turning them into workable position-position-velocity cubes 
# (The cubes are 3D, but the moment maps/continuum images will be 2D and SC will also work for them)

from spectral_cube import SpectralCube    
from spectral_cube import LazyMask
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
print('spectral_cube file path',spectral_cube.__file__)

# Need this for projection of the cubes

from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)

# Useful for doing analysis

import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import matplotlib.colors as colors
from matplotlib import pyplot as plt
import scipy
import numpy as np                          
from matplotlib.patches import Ellipse
print(np.__version__,"Numpy")
from scipy.optimize import curve_fit
from scipy.optimize import leastsq

# astrodendro is a key package, it allows a quick way to identify structures

import astrodendro 
from astrodendro.analysis import PPVStatistic # Takes statistics of PPV structures

print("astrodendro_file:", astrodendro.__file__)

# The radio_beam library allows you to check/change teh interferometric properties of the beam.

import radio_beam

# Garbage collection

import gc

# Suppress warnings we don't care about:

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

    
#     

from tqdm import tqdm

#

gal="GC"

dist_cmz = 8.178*10**-3*u.Mpc
   
    

if(os.path.exists("./Result Files")):
    print("Results will be saved to Directory ./Result Files")
else:
    %mkdir "./Result Files"
    print("Created new directory ./Result Files where the results will be saved")
    
# If you need to interact with teh plots, use widget, otherwise this runs better

%matplotlib inline
#%matplotlib widget 


/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16
/home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
/home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
1.23.1 Numpy
Results will be saved to ./Result Files


# Functions

Make general functions for each process so I can call them simply.
The only things I need to change are the input files and their properties

# Pointing Information

In [ ]:
# Pointing_Information


# Pointing_Information an important variable that will need to be defined for each source
# It is a dictionary containing all the metadata and pointing information. This is important because the PPVstatistic needs it to be in a specific format
#
# eg Pointing_Information:  
'''

Pointing_Information = {}

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "HCN_J1-0.cube.fits" #the name of the initial SC file.
Pointing_Information["File_Descriptor"] = "NGC_253_HCN_J1-0_"
Pointing_Information["target"] = "NGC253"#or the cmz
Pointing_Information["center"] = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs') #the center of NGC253
#I use center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') for the cmz center
desired_beam_size = 4.3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 3.5*u.Mpc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=33*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=78*u.deg
Pointing_Information["target_velocity"]=250*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"]= 3.3*u.km/u.s
ovs = 3 #how much do you desire to oversample the beam by
desired_fov = [360*u.pc,70*u.pc]


#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header

try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=header["RESTFREQ"]#            
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=header["RESTFRQ"]#    

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 
    
beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec)


Pointing_Information["original_BMAJ"]=beam_major
Pointing_Information["original_BMIN"]=beam_minor
Pointing_Information["original_BMAJ_pc"]=beam_major.to(u.rad)*Pointing_Information['distance']
Pointing_Information["original_BMIN_pc"]=beam_minor.to(u.rad)*Pointing_Information['distance']
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size


#this accounts for elliptical beams:            
Pointing_Information['original_pixel_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_pixel_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
Pointing_Information['original_spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']#convert to pc using the distance

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Pointing_Information['original_beam_oversampling_MAJ'] = beam_major/average_pixel
Pointing_Information['original_beam_oversampling_MIN'] = beam_minor/average_pixel
Pointing_Information['desired_beam_oversampling'] = ovs
Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target'],Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_fov,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######


#Cube_Information. This needs to be updated every time a reduction occurs on the cube. By the end of file 1 which does all the reprojection, this will not change.
#The pointing information will not change, so I'll start with a copy of that:

Cube_Information = copy.deepcopy(Pointing_Information)

#For example, after doing the reprojection, I need to load: 


Current_Cube_Name = Pointing_Information["File_Descriptor"] + str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'++str(i)+'.fits'
Cube_Information["File_Name"]=Current_Cube_Name

sc = SpectralCube.read(Current_Cube_Name)
header = sc.header

pc_per_pixelx = abs(header["CDELT1"]*Cube_Information["CUNIT"].to(u.rad))*Pointing_Information['distance']/u.pix #convert degrees to size using the distance
pc_per_pixely = abs(header["CDELT2"]*Cube_Information["CUNIT"].to(u.rad))*Pointing_Information['distance']/u.pix #convert degrees to size using the distance

Cube_Information["pixel_scale_x"]=pc_per_pixelx
Cube_Information["pixel_scale_y"]=pc_per_pixely

Cube_Information['data_unit'] =sc[0][0][0].unit# or just use sc.header['BUNIT']

Cube_Information['arc_per_pix_y'] =  abs(header["CDELT1"]*CUNIT).to(u.arcsec)/u.pix 
Cube_Information['arc_per_pix_x'] =  abs(header["CDELT2"]*CUNIT).to(u.arcsec)/u.pix

beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec) #if these two are different, something went wrong in the reprojection
Cube_Information['beam_major'] =  beam_major
Cube_Information['beam_minor'] =  beam_minor

beam_area_ratio = beam_minor*beam_major/Cube_Information['arc_per_pix_y']/Cube_Information['arc_per_pix_x']*1.13309#beam_area_ratio = Cube_Information['beam_minor']*Cube_Information['beam_major']/Cube_Information['arc_per_pix_y']/Cube_Information['arc_per_pix_x']#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
Cube_Information['beam_area_ratio']=beam_area_ratio #This is important for finding the minimum number of pixels a structure can have, or for calculating column densities

#this accounts for elliptical beams:            
Cube_Information['spatial_scale_x'] = abs(header["CDELT1"])*CUNIT/u.pix
Cube_Information['spatial_scale_y'] = abs(header["CDELT2"])*CUNIT/u.pix
Cube_Information["velocity_scale"] = abs(header["CDELT3"])*u.km/u.s#the cube should be in u.km/u.s instead of frequency on the z axis

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Cube_Information['beam_oversampling'] = beam_minor/average_pixel
Cube_Information['desired_beam_oversampling'] = Pointing_Information['desired_beam_oversampling']

Cube_Information["wcsu"]=sc.wcs






'''

#make a function to do this easily

def Update_Cube_Information(Pointing_Information,Current_Cube_Name):

    Cube_Information = copy.deepcopy(Pointing_Information)

    Cube_Information["File_Name"]=Current_Cube_Name
    try:
        del sc
    except:
        pass
    sc = SpectralCube.read("Spectral Cubes/"+Current_Cube_Name)
    header = sc.header

    pc_per_pixelx = abs(header["CDELT1"]*Cube_Information["CUNIT"].to(u.rad))*Pointing_Information['distance']/u.pix #convert degrees to size using the distance
    pc_per_pixely = abs(header["CDELT2"]*Cube_Information["CUNIT"].to(u.rad))*Pointing_Information['distance']/u.pix #convert degrees to size using the distance

    Cube_Information["pixel_scale_x"]=pc_per_pixelx
    Cube_Information["pixel_scale_y"]=pc_per_pixely

    Cube_Information['data_unit'] =sc[0][0][0].unit# or just use sc.header['BUNIT']

    Cube_Information['arc_per_pix_y'] =  abs(header["CDELT1"]*CUNIT).to(u.arcsec)/u.pix 
    Cube_Information['arc_per_pix_x'] =  abs(header["CDELT2"]*CUNIT).to(u.arcsec)/u.pix

    beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
    beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec) #if these two are different, something went wrong in the reprojection
    Cube_Information['beam_major'] =  beam_major
    Cube_Information['beam_minor'] =  beam_minor

    beam_area_ratio = beam_minor*beam_major/Cube_Information['arc_per_pix_y']/Cube_Information['arc_per_pix_x']*1.13309#beam_area_ratio = Cube_Information['beam_minor']*Cube_Information['beam_major']/Cube_Information['arc_per_pix_y']/Cube_Information['arc_per_pix_x']#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
    Cube_Information['beam_area_ratio']=beam_area_ratio #This is important for finding the minimum number of pixels a structure can have, or for calculating column densities

    #this accounts for elliptical beams:            
    Cube_Information['spatial_scale_x'] = abs(header["CDELT1"])*CUNIT/u.pix
    Cube_Information['spatial_scale_y'] = abs(header["CDELT2"])*CUNIT/u.pix
    Cube_Information["velocity_scale"] = abs(header["CDELT3"])*u.km/u.s#the cube should be in u.km/u.s instead of frequency on the z axis

    average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

    Cube_Information['beam_oversampling'] = beam_minor/average_pixel
    Cube_Information['desired_beam_oversampling'] = Pointing_Information['desired_beam_oversampling']

    Cube_Information["wcsu"]=sc.wcs
    
    Cube_Information['FOV']= Crop_Around_Center(sc,Cube_Information['target_image_rotation'],Cube_Information['center'],Cube_Information['desired_FOV'],Cube_Information['distance'])[1] #returns [the cropped sc, the current fov, the desired fov]
    Cube_Information['desired_FOV']=Pointing_Information['desired_FOV']
    
    return Cube_Information

# Reprojection function

To align the files across and get the same beam size and FOVs along different observations

In [ ]:

#Spatial reprojection 

# File = the file name of the SC you want to reproject
# Prime_Beam = the pc size of the beam. When comparing a line in two different sources, i match the beam sizes across the sources

# Pointing_Information= see earlier
# Cube_Information= see earlier
# i_step = the amount of velocity channels for each step in the reprojection. lower this if your computer lags a lot.

def Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[False,[0,0]*u.deg,[0,0]*u.deg]):
    
    File = Cube_Information["File_Name"]
    Prime_Beam = Pointing_Information["desired_beam_size"]
    Gal = Pointing_Information["target"]
    ovs = Pointing_Information["desired_beam_oversampling"]
    FOV = Pointing_Information["desired_FOV"]
    distance=Pointing_Information["distance"]
    target_velocity= Pointing_Information["target_velocity"]
    center = Pointing_Information["center"]
    rotation_angle = Pointing_Information["target_image_rotation"]
    Line_Name = Pointing_Information["File_Descriptor"]

    
    # These are related to error correction the original CMZ cubes because they have an error with their longitude values going from 360->0 through the center
    
    Force_Origins=Force_Origin[0]
    Force_Value_x=Force_Origin[1]
    Force_Value_y=Force_Origin[2]
    
    #Load observation cube
    
    scB = SpectralCube.read(File) 
    scB.allow_huge_operations=True
    scB = scB.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s in case they are in Hz
    
    #Determine the resolution based on the oversample factor
    
    beam_deg =  ((Prime_Beam/(distance))*u.rad).to(u.deg)#deg corresponding to the desired beam size
    
    # Check the coordinate systems
    
    vel,RA,Dec = scB.world[:,0,0]
    
    # Need to break it up into 30-wide vel slices to do the reprojection (ram-draw too high otherwise)  
    
    for i in range(int(len(scB)/i_step) +1):
        
        try:
            print('begin step:',i,"of",int(len(scB)/i_step) +1)
            
            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOV[0].value)+"x"+str(FOV[1].value)+'pc_'+'reprojected.fits'
            
            sc = scB[i*i_step:i*i_step+i_step]
            sc = sc.spectral_slab(target_velocity- 251*u.km/u.s, target_velocity+ 251*u.km/u.s)  # Crop out velocities we don't care about  
            sc.allow_huge_operations=True
            
            print('start beam convolution')

            try:
                #Make a circular beam to convolve the image to.
    
                beam = radio_beam.Beam(major=beam_deg, minor=beam_deg, pa=0*u.deg)
                sc = sc.convolve_to(beam)
            
            except:
                print("The initial image has no beam because it's not interferometric, so one will be created using the pixel size as the initial beam size")
                
                cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
                cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
                if(cdelt_x>cdelt_y):
                    majorBase=cdelt_x
                    minorBase=cdelt_y
                elif(cdelt_x<cdelt_y):
                    majorBase=cdelt_y
                    minorBase=cdelt_x
                elif(cdelt_x==cdelt_y):
                    majorBase=cdelt_x
                    minorBase=cdelt_x
                    
                BaseBeam = radio_beam.Beam(major=majorBase, minor=minorBase, pa=0*u.deg)

                sc = sc.with_beam(BaseBeam)
                beam = radio_beam.Beam(major=beam_deg, minor=beam_deg, pa=0*u.deg)

                sc.allow_huge_operations=True
                #Requires me to edit convolve.py and set allow_huge =True
                #If this fails, go edit that file in your repository.
                sc = sc.convolve_to(beam)
                
            print('convolve end\n')
            
            
            
            #Mask the pixels outside the fov
            
            #returns: cropped_sc,orig_fov,FOV
            print('fov crop start \n')
            cropped_sc = Crop_Around_Center(sc,rotation_angle,center,FOV,distance)[0]
            del sc
            sc = cropped_sc
            del cropped_sc
            print('fov cropped\n')
            
            
            #
            #prepare a header for the reprojection
            #
            
            reheader = copy.deepcopy(sc.hdu.header)
            

            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

            #find out which direction the cube is read, left to right or right/to left. (in terms of RA/DEC). Then, do the same for up and down
            
            if sc.header['cdelt1']>0:
                pix_x    = (beam_deg/ovs).to(u.degree).value
                origin_x = sc.longitude_extrema[0].to(u.degree).value
                
                if(Force_Origins):
                    origin_x = Force_Value_x[0]#358.6

            else:
                pix_x    = -1.*(beam_deg/ovs).to(u.degree).value
                origin_x = (sc.longitude_extrema[1]).to(u.degree).value
                
                if(Force_Origins):
                    origin_x = Force_Value_x[1]#.9

            if sc.header['cdelt2']>0:
                pix_y    = (beam_deg/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[0].to(u.degree).value
                
                if(Force_Origins):
                    origin_y = Force_Value_y[0]#-.6
                    
            else:
                pix_y    = -1.*(beam_deg/ovs).to(u.degree).value
                origin_y = sc.latitude_extrema[1].to(u.degree).value
                
                if(Force_Origins):
                    origin_y = Force_Value_y[1]#.6
                    

            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            
            if(Force_Origins):
                npix_x   =int(np.diff(Force_Value_x)/np.abs(pix_x).value)
                npix_y   =int(np.diff(Force_Value_y)/np.abs(pix_y).value)
                
            #Correct the header to the expected pixels for the new res

            reheader['cdelt1'] = pix_x
            reheader['cdelt2'] = pix_y

            reheader['naxis1'] = npix_x
            reheader['naxis2'] = npix_y

            reheader['crval1'] = origin_x
            reheader['crval2'] = origin_y

            reheader['crpix1'] = 0
            reheader['crpix2'] = 0
            
            if(str(sc.wcs).find("GLON")!=-1):
                reheader['CTYPE1'] = "GLON-SIN"
                reheader['CTYPE2'] = "GLAT-SIN"
                
            # remove these things that confuse the reprojection since we won't be using them
            try:
                del reheader['lonpole']
                del reheader['latpole']
                del reheader['wcsaxes']# Dont need these anymore, maybe?
                if(str(sc.wcs).find("GLON")!=-1):

                    del reheader['LBOUND1']
                    del reheader['LBOUND2']
                    del reheader['LBOUND3']
                    del reheader.cards['LBOUND1']
                    del reheader.cards['LBOUND2']
                    del reheader.cards['LBOUND3']

                    reheader['LBOUND1']=0
                    reheader['LBOUND2']=0
                    reheader['LBOUND3']=0
            except Exception as e:
                print(e)
                print("-"*60)
                traceback.print_exc(file=sys.stdout)

            # regrid cube to target pixel size

            print('start reprojection\n')
            print('check max SC value:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
            
            sc2 = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True) # Had to change reproject.py so it deletes output.np before making a new one

            del sc # save space

            # make a new cube with the reprojcted data (remove all the logs from the old cube)
            
            new = SpectralCube(data=sc2.hdu.data,wcs =WCS(sc2.header),header=sc2.header,mask=sc2.mask)
            new.allow_huge_operations=True
            new = new*sc2[0][0][0].unit
            
            #do this because scs dont like being modified
            del sc2
            sc2 = new
            del new
            
            print('\nend reprojection\n')
            print('check max SC value:',np.nanmax(sc2),"SC shape:", np.shape(sc2))#These should ALSO be a non zero float and the shape of the cube (30,~1000,~1000)

            sc = sc2
            del sc2
            sc.allow_huge_operations=True

            #Mask the pixels outside the fov again after the reprojection to get rid of nan created pixels
            
            print('fov crop start 2 \n')
            cropped_sc = Crop_Around_Center(sc,rotation_angle,center,FOV,distance)[0]
            del sc
            sc = cropped_sc
            del cropped_sc
            print('fov cropped 2\n')
            
            # Write the intermediary cubes that will be spliced together
            
            sc.write("Spectral Cubes/"+str(str(i)+"_"+Cube_Name_Save),overwrite=True)

        except Exception as e:
            print(e)
            print("Failed (unless this says attempt to get argmin of empty sequence)")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)

            


In [ ]:

#############
# Crop_Around_Center
#############


#This function crops out things outside the rectangle where the actual disk lies.
#The disk may be rotated, so this will be a rotated rectangle.
#Without a circular beam, this will be off, but I only use it after I circularize the beam.


#returns: cropped_sc,orig_fov,FOV

def Crop_Around_Center(sc_for_cropping,rotation_angle,center,desired_fov,distance):

    
    
    r,c,FOV,d = rotation_angle,center,desired_fov,distance
    
    
    cdelt_x = u.Quantity(str(np.abs(sc_for_cropping.header['cdelt1']))+sc_for_cropping.header['cunit1'])
    cdelt_y = u.Quantity(str(np.abs(sc_for_cropping.header['cdelt2']))+sc_for_cropping.header['cunit2'])
    center_ra_pix,center_dec_pix = [int(sc_for_cropping.wcs[:][:][0].world_to_pixel(c)[0]),int(sc_for_cropping.wcs[:][:][0].world_to_pixel(c)[1])]
    PixFov = [int((FOV[0].to(u.pc)/(cdelt_x.to(u.rad)*d.to(u.pc))).value/2),int((FOV[1].to(u.pc)/(cdelt_x.to(u.rad)*d.to(u.pc))).value/2)] #they'll be in pixels, but I only need the int

    print("Center:",c,"Pixel center:",center_ra_pix,center_dec_pix,"Pixel FOV:",PixFov)
    
    pixels = np.zeros(np.shape(sc_for_cropping))           
    
    print("cropping cube. rotation:",r,"center:",center,"crop to:",desired_fov)
    
    if(r!=0*u.deg):
        #to save time
        r_rad=r.to(u.rad).value
        #Find the pixels that are outside the rectangular FOV
        for lmj in range(np.shape(sc_for_cropping)[1]):
            for lmk in range(np.shape(sc_for_cropping)[2]):

                #The hypotenuse
                hypo = np.sqrt(((lmj-center_dec_pix)**2) + (lmk-center_ra_pix)**2)

                if (lmj-center_dec_pix!=0):
                    ang = np.arctan(abs(lmk-center_ra_pix)/abs(lmj-center_dec_pix))#*u.rad#Find angle to the center
                else:
                    ang = np.pi/2#*u.rad
                if(lmk>center_ra_pix and lmj>center_dec_pix):
                    ang*=-1
                elif(lmk<center_ra_pix and lmj<center_dec_pix):
                    ang*=-1
                elif(lmk>center_ra_pix and lmj<center_dec_pix):
                    if ang > (np.pi/2-r_rad):

                        ang= -r_rad+(np.pi-r_rad)-ang#coming from the opposite end of the ra axis now, but projecting still to 33 deg from north.
                    else:
                        pass


                up_pixels = abs(hypo*np.cos(abs(r_rad+ang)))
                side_pixels = abs(hypo*np.sin(abs(r_rad)+ang))

                #Check if the pixels are inside the FOV
                if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                    for lmi in range(np.shape(sc_for_cropping)[0]):
                        pixels[lmi][lmj][lmk] = 1  # keep this pixel
                
    else:
        
        # If the image is not rotated, the FOV is just a rectangle
        
        # Find all pixels in the fov
        
        for lmj in range(np.shape(sc_for_cropping)[1]):
            for lmk in range(np.shape(sc_for_cropping)[2]):

                up_pixels = abs(lmj-center_dec_pix)#Should not be over the fov in the upwards direction (relative to 0 degrees)
                side_pixels = abs(lmk-center_ra_pix)#Should not be over the fov in the side-side direction (relative to 0 degrees)

                if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                    for lmi in range(np.shape(sc_for_cropping)[0]):
                        pixels[lmi][lmj][lmk] = 1  # keep this pixel
                        
    # Mask teh pixels outside the fov

    bp = np.where(pixels!=1)
    scCopy = sc_for_cropping.hdu
    scCopy.data[bp]=np.nan
    scP = SpectralCube.read(scCopy)
    del scCopy
    del bp
    #Get right size by removing the nan pixels
 
    scP.allow_huge_operations=True
    datn = scP.hdu.data
    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(datn[0,:,:])[0]):

        # Go through a slice of the cube and find the first pixels with rael data
        
        # After this is done these will be non zero so break the loop
        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            
            break
            
        for lmj in range(np.shape(datn[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(datn[0,lmi,:])>0 or np.nanmean(datn[0,lmi,:])<0):
                    sx=lmi

            if(sy==0):
                if(np.nanmean(datn[0,:,lmj])>0 or np.nanmean(datn[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(datn[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])>0 or np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(datn[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break

    sc1 = scP[:,sx:ex,sy:ey]
    scP_Hdu=sc1.hdu
    
    
    # Also check if there are zeroes in place of nan values, which is done on some cubes
    
    zeros=((scP_Hdu.data[:,:,:]==0))
    bp = np.where(zeros)
    scP_Hdu.data[bp]=np.nan
    scCopy = SpectralCube.read(scP_Hdu)
    del scP_Hdu
    
    # Make the final cropped cube

    cropped_sc = SpectralCube.read(scCopy.hdu)
    
    del scCopy
    
    data=sc_for_cropping.hdu.data
    
    
    ######## find the original FOV
    
    fp_dec,lp_dec,fp_ra,lp_ra=0,0,0,0
    for lmj in range(np.shape(sc_for_cropping)[1]):
        for lmk in range(np.shape(sc_for_cropping)[2]):
                if (data[0][lmj][lmk]>0 or data[0][lmj][lmk]<0):
                    pass
                else:
                    fp_dec=lmj#the first upwards pixel with data
                    break
        if (fp_dec != 0):
            break
    for lmj in range(np.shape(sc_for_cropping)[1]):
        for lmk in range(1,np.shape(sc_for_cropping)[2]):
                if (data[0][lmj][np.shape(sc_for_cropping)[2]-lmk]>0 or data[0][lmj][np.shape(sc_for_cropping)[2]-lmk]<0):
                    pass
                else:
                    lp_dec=np.shape(sc_for_cropping)[2]-lmk-1#the last upwards pixel with data
                    break
        if (lp_dec != 0):
            break
    for lmk in range(np.shape(sc_for_cropping)[1]):
        for lmj in range(np.shape(sc_for_cropping)[2]):
                if (data[0][lmj][lmk]>0 or data[0][lmj][lmk]<0):
                    pass
                else:
                    fp_ra=lmj#the first sideways pixel with data
                    break
        if (fp_ra != 0):
            break
    for lmk in range(np.shape(sc_for_cropping)[1]):
        for lmj in range(1,np.shape(sc_for_cropping)[2]):
                if (data[0][np.shape(sc_for_cropping)[1]-lmj][lmk]>0 or data[0][np.shape(sc_for_cropping)[1]-lmj][lmk]<0):
                    pass
                else:
                    lp_ra=np.shape(sc_for_cropping)[1]-lmj-1#the last sideways pixel with data
                    break
        if (lp_ra != 0):
            break
            
            
    if(r!=0*u.deg):
        if(lp_ra-fp_ra>lp_dec-fp_dec):
            disk_pixels=(lp_dec-fp_dec)/np.sin(r_rad)
            jet_pixels=(lp_dec-fp_dec)/np.sin(np.pi/2 - r_rad)
        else:
            disk_pixels=(lp_ra-fp_ra)/np.sin(r_rad)
            jet_pixels=(lp_ra-fp_ra)/np.sin(np.pi/2 - r_rad)
    else:
        disk_pixels=lp_ra-fp_ra
        jet_pixels=lp_dec-fp_dec
        
    orig_fov=[0,0]            
    orig_fov[0] = np.round(((disk_pixels*(cdelt_x.to(u.rad)*d))*2/u.rad).to(u.pc),1) #the fov across the disk
    orig_fov[1] = np.round(((jet_pixels*(cdelt_x.to(u.rad)*d))*2/u.rad).to(u.pc),1) #the fov coming up from the disk
    
    
    
    
    print("cropped cube from",orig_fov,"to:",desired_fov)
    
    return cropped_sc,orig_fov,FOV

In [ ]:

# Splice the partwise cubes back together:
    
def Splice_vels(Cube_Name_Load):
    
    for i in range(2000):
        try:
            Cube_Name_Load_p = str(i)+"_"+Cube_Name_Load
            Cube_Name_Save = Cube_Name_Load

            sc=SpectralCube.read(("Spectral Cubes/"+Cube_Name_Load_p)) 
            print("Loaded",Cube_Name_Load_p)
            
            
            # Define a header that we will form into the new header for the spliced cube
            if i == 0:
                reheader = sc.header
                rewcs = sc.wcs

            if i == 0:
                scW=SpectralCube.read(("Spectral Cubes/"+Cube_Name_Load_p))      
                mask = scW.mask.include() # Need to create a mask because it doesn't get splcied
                
            else:
                if i == 1:
                    scW = np.concatenate((scW[:].hdu.data,sc[:].hdu.data),dtype = type(sc))
                    mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))
                else:
                    scW = np.concatenate((scW[:],sc.hdu.data[:]),dtype = type(sc))
                    mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))

        except Exception as e:
            print(e)
            break
            
    # This only matters for formatting reasons:
    def duh(lol):
        gp = np.where(lol!=np.nan)
        lol[gp]=True
        return lol # Anywhere that has data will be unmasked
    
    reheader["NAXIS3"] = len(scW)
    Full_Mask = LazyMask(function = duh,data = mask, wcs = rewcs)
    
    
    scWsc = SpectralCube(data = scW,wcs = rewcs, header = reheader, mask = Full_Mask)# The spliced cube

    scWsc.allow_huge_operations=True
    scWsc = scWsc*sc[0][0][0].unit#Add unit back in
    del sc


    scWsc.write("Spectral Cubes/"+Cube_Name_Save,overwrite=True)



In [ ]:
###
# Velocity reprojection
###
        
# Smooth the data (gaussian) to match the velocity resolution of other cubes

def Repo_Velocity(scp,Cube_Name_Save,Cube_Information):
    
    print("Start velocity reprojection of" ,Cube_Name_Save)
    
    vel_prime = Cube_Information["desired_velocity_resolution"] 
    target_velocity = Cube_Information["target_velocity"]
    Initial_vel = u.Quantity(str(np.abs(scp.header['cdelt3']))+scp.header['cunit3']) # The current velocity resolution
    restfreq = Pointing_Information['restfreq']
    
    # Gaussian width = sqrt( new_vel^2 -  old_vel^2)
    
    G_width = np.sqrt((vel_prime.to(u.km/u.s)).value**2-(Initial_vel.to(u.km/u.s)).value**2) 
    
    vel = np.arange((target_velocity - 251*u.km/u.s).value, (target_velocity + 251*u.km/u.s).value,vel_prime.to(u.km/u.s).value)*u.km/u.s #make the new velocity axis
    scWsc_copy = scp
    
    # the factor converting the gaussian width to its FWHM
    fwhm_factor = np.sqrt(8*np.log(2))

    scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq) # Make sure it has the right rest frequency
    
    # The reprojected velocity must be larger than the initial velocity or else this gives an error
    scWsc_copy = scWsc_copy.spectral_smooth(Gaussian1DKernel(G_width/fwhm_factor))#Preserves information from the pixels lost in downsampling
    
    print("Smoothed to Gaussian Kernel of width",G_width/fwhm_factor)

    scWsc_copy = scWsc_copy.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 251 range  

    
    Cube_Name_Save_new = Cube_Name_Save[0:len(Cube_Name_Save)-6]+"_"+str(vel_prime.value)+'_vel_res_'+'.fits'
    
    
    scWsc_copy.write("Spectral Cubes/"+Cube_Name_Save_new,overwrite=True)
    
    print("Wrote reprojected cube to","Spectral Cubes/"+Cube_Name_Save_new)
    
    gc.collect()
    del scWsc_copy

    gc.collect()######################################################################



    

In [ ]:
# after the reprojection, there may be duplicated data from the fact that it just fills in stuff 
# expecting new data to be there. This fixes it by checking to see if it matches other data, then removes
# the repeated slices. in other words, it may be set to reproject to 0-500 km/s through the velocity channels
# but there may only be data between 100-400, so it will fill it in with copied data channels. This will check for those.

In [ ]:
# Fix reprojected repeated pixels

# All these parameters only matter here because they are in the cube name
# Line_Name = Line Name, Beam_Size=beam size, FOV=field of view, min_vel =  velocity resolution

def Remove_Repeated_Pixels(Cube_Name_Load):
    
    Cube_Name_Save = "Cropped_"+Cube_Name_Load

    scRRP = SpectralCube.read("Spectral Cubes/"+Cube_Name_Load).with_spectral_unit(u.km/u.s,velocity_convention="radio")
    
    
    sp=0 #starting pixel with real data (tbd)
    
    for lmi in range(len(scRRP)):
        #Check to see if the slice has been repeated by the interpolation function
        if(np.round(np.nanmean(scRRP[lmi].hdu.data),9)==np.round(np.nanmean(scRRP[lmi+1].hdu.data),9)):

            sp = lmi+1
        else:
            print("Good data starting from channel",sp,"; start has been cropped to that channel")
            break
            
    l = len(scRRP)-1
    ep=l
    for lmi in range(l):
        #Cehck again, starting from the end this time
        if(np.round(np.nanmean(scRRP[l-lmi].hdu.data),9)==np.round(np.nanmean(scRRP[l-lmi-1].hdu.data),9)):
            ep = l-lmi-1

        else:
            print("Good data ending at channel",ep,"; end has been cropped to that channel")
            break

    #sp,ep, These are the start and stop slices where the actual unique data resides

    scRRP.allow_huge_operations=True
    scRRP = scRRP[sp:ep]



    scP_Hdu=scRRP.hdu
    zeros=((scP_Hdu.data[:,:,:]==0))
    bp = np.where(zeros)
    scP_Hdu.data[bp]=np.nan
    scRRP = SpectralCube.read(scP_Hdu)



    scRRP = scRRP.to(u.K)

    scRRP.write(("Spectral Cubes/"+Cube_Name_Save),overwrite=True)
    del scRRP
    del scP_Hdu

    print("Cropped cube saved as ","Spectral Cubes/"+Cube_Name_Save)

# PPV statistics

 This function uses the dendrogram as an input and calculates all the quantities
 such as size, linewidth, Column Density, RMS velocity (velocity dispersion, a measure 
 of turbulence), Luminosity (using the distance and the flux), Distance to structure (only
 matters for the CMZ), the Moment0_Flux, and the V_RMS_error
 
 


 
 These are all the things calculated in the Shetty 2012 paper

 To exclude incomplete, unresolvable, or otherwise nan strucutres, we use some exclusion properties:
 1. Don't allow any structure under 3 pc in size (the beam size)
 2. Don't allow any structure



Returns np.array(SizeA), np.array(SigmaA), np.array(CDA) ,np.array(LuminA) ,np.array(SIDS) , np.array(MOM0_FLUX) , np.array(Distances), np.array(V_rms_err) for each structure in the dendrogram and returns them in the form [[][]], where [Leaves][Branches] are the different kinds of structures in the dendrogram



In [ ]:

#Continuum is in Jansky/Beam, Line data should have the unit specified in the metadata as 'data_unit'

#Dendrogram=the computed dendorgam, 
#Line Data= the actual np.array of the cube's data
#
#Pointing_Information = defined above

#Max_Size=the maximum allowed size for a strcutre in u.pc
            
#beam_size=the size of the beam in u.pc
#beam_req= 1/(how much you are willing to oversample the beam)
#Trunks= True/False if you want/dont want to include the dendrogram trunks as branches

# Note: The column density information is not yet fully bug tested, do not use it as gospel
                        
def Dendro_Arrays(Dendrogram,LineData,DataVel,ContData,Pointing_Information,beam_size=999,beam_req = 1/3,Trunks=True,max_size=18,edge_cases=False):
    
    #init all the return arrays:                    
    SizeA,SigmaA,LuminA,CDA,SIDS,MOM0_FLUX,Distances,V_rms_err = [[],[],[],[]],[[],[],[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]]
            
    metadata=Pointing_information    
            
    dist_val=metadata['distance'].to(u.pc).value  #convert to the value in pc

    center = Pointing_Information["center"]
                        
    d_copy= Dendrogram #copy the dendrogram

    
    center_ra_pix,center_dec_pix = int(metadata['wcsu'][:][:][0].world_to_pixel(center)[0]),int(metadata['wcsu'][:][:][0].world_to_pixel(center)[1])
    
                        
    sliced= LineData[6]
    CubeShape = np.shape(sliced) #the shape of a slice of the cube
    DataShape=[[0,0],[0,0]]#We will find the part of the cube that actually has data

    for lmi in range(CubeShape[0]):
        allData=np.nansum(sliced[lmi])
        if(allData>0 or allData<0):
            DataShape[0][0] = lmi+3
            break
    for lmi in range(CubeShape[0]):
        allData=np.nansum(sliced[CubeShape[0] - lmi -1])
        if(allData>0 or allData<0):
            DataShape[0][1] = CubeShape[0] - lmi -3
            break
    for lmi in range(CubeShape[1]):
        allData=(sliced[DataShape[0][0],lmi])
        if(allData>0 or allData<0):
            DataShape[1][0] = lmi+3
            break
    for lmi in range(CubeShape[1]):
        allData=(sliced[DataShape[0][0],CubeShape[1] - lmi -1])
        if(allData>0 or allData<0):
            DataShape[1][1] = CubeShape[1] - lmi -3
            break
    for t in Dendrogram.all_structures: 

        I = t.indices() #the pixel values in x, y of the structure
        Cont = True
        if t.is_branch:
                if t.parent==None:
                    
                    if(Trunks):
                        Cont = True
                    else:
                        Cont = False
                else:
                    Cont=True
        
        if edge_cases ==False:
            #the milky way data is flat soo the calucation is easier
            if gal =="GC":
            
            
                for lmi in range(len(I[0])):
                        
                    #cehck if there are any nan values nearby or any edges of the cube nearby
                        
                    if(I[1][lmi]<=DataShape[0][0] or I[1][lmi]>=DataShape[0][1] or I[2][lmi]<=DataShape[1][0] or I[2][lmi]>=DataShape[1][1]):
                        Cont=False
                        break
                        
                    elif(I[1][lmi]<=1 or I[1][lmi]>=CubeShape[0] or I[2][lmi]<=1 or I[2][lmi]>=CubeShape[1]):
                        Cont=False
                        break
                        
            else:          

                for lmi in range(len(I[0])):
                    NansNE=0
                    NansSE=0
                    NansNW=0
                    NansSW=0
                    Length = 5
                    for lmj in range(Length):
                        #Check four 45 degree prongs from each point and see if they have nans. If that happens its too close to the boundary or the data is bad
                        try:

                            if(sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]<0 ):
                                pass
                            else:
                                NansSE+=1
                            if(sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]<0 ):
                                pass
                            else:
                                NansSW+=1
                            if(sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]<0 ):
                                pass
                            else:
                                NansNW+=1
                            if(sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]<0 ):
                                pass
                            else:
                                NansNE+=1
                        except:
                            #only fails if the I goes close to the boundary of the cube and tries to get a pixel outside the cube
                            Cont = False
                            break
                    #count the number of nans nearby:
                        
                    if(NansNE>2 or NansNW>2 or NansSE>2 or NansSW>2):
                        Cont=False
                        break


        if(Cont):
            s = PPVStatistic(t,metadata=metadata) #calcuate the properties of the structure.this function is from the astrodendro library
            s_radius = s.radius #Give the size in degrees
            s_v_rms = s.v_rms #in u.km/u.s
                        
            Parsec_Size = (float(s_radius*np.pi/180*dist_val/u.deg)) #Convert to parsecs using the distance in Pc
            #also check to make sure the size is greater than 1/3 the beam. any less and it will be too much noise
            #and make sure the rms velocity is non-zero. otherwise, its not a 3d structure
            if(Parsec_Size<max_size and Parsec_Size>beam_size*1/3 and s_v_rms>.01*u.km/u.s):
            
            

                nproj_pix=len(set(zip(*tuple(I[i] for i in [1,2]))))
                v_IWM = np.nansum(LineData[I]*(DataVel[I[0]])/u.km*u.s)/np.nansum(LineData[I])
                sig_Sh = np.sqrt(np.nansum(LineData[I]*((DataVel[I[0]])/u.km*u.s-v_IWM)**2)/np.nansum(LineData[I])) 
                
                #The flux from the continuum
                #Convert to Jansky from Jansky per beam:
                if(ColD ==True):
                    Cont_Flux=0

                    proj = tuple(set(zip(*tuple(I[i] for i in [1,2]))))
                    for lmi in range(len(proj)):

                        Cont_Flux+=ContData[proj[lmi]]
                    Cont_Flux=Cont_Flux/(metadata['beam_area_ratioc']*(2*np.sqrt(2*np.log(2))))*u.pix**2*u.beam/u.beam*u.Jy#SHould be input as Jansky /beam and will be converted to Jansky, then to unitless. The beam is changed from FWHM to Gaussian
                    Dust_Column = Flux_to_Mass(Cont_Flux)*Num_per_kg/((s_radius*np.pi/180*dist_cmz.value/u.deg)**2*(3.086*10**24)**2)/np.pi*(1.989*10**30*u.kg/u.M_sun)/u.kg
                    
                else:
                    Dust_Column=0
                if(str(Dust_Column) == str(np.nan) or str(Dust_Column)==str(np.inf)):
                    Dust_Column=0
                lum = Flux_to_Lum(s.flux)
                s_flux = s.flux

                Index = tuple(I[i] for i in [0,1,2])
                K_Km_s_Flux=np.nansum(LineData[Index]*metadata["velocity_scale"])#Find the total flux from the structures in K km/s, assuming the input data is in K as it should be, 
                
                
                
                
                Distance = np.sqrt((float(s.x_cen/u.pix)-center_ra_pix)**2+(float(s.y_cen/u.pix)- center_dec_pix)**2)*metadata['spatial_scale']*np.pi/180*dist_cmz.value*10**6/u.deg#pc dist from barycenter
                
                
                V_err= Get_V_rms_err(dend1=d_copy,idx=int(t.idx),struct=t,m=m,NF=1,iterations=5,metadata=metadata)
                
                
                if(t.is_leaf):

                    SizeA[0].append((float((s_radius*np.pi/180*dist_val/10**6/u.deg)))) #define size as astrodendro
                    SigmaA[0].append((float(s_v_rms/u.km*u.s)))#
                    CDA[0].append(float(Dust_Column))
                    LuminA[0].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[0].append(float(t.idx))
                    MOM0_FLUX[0].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[0].append(float(Distance))
                    V_rms_err[0].append(float(V_err))
                if(t.is_branch	):

                    SizeA[1].append((float((s_radius*np.pi/180*dist_val/10**6/u.deg)))) #define size as astrodendro
                    SigmaA[1].append((float(s_v_rms/u.km*u.s)))#
                    CDA[1].append(float(Dust_Column))
                    LuminA[1].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[1].append(float(t.idx))
                    MOM0_FLUX[1].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[1].append(float(Distance))
                    V_rms_err[1].append(float(V_err))
                del s
                    
                    
                    
    SizeA[0] = np.array(SizeA[0],dtype=type(1.))
    SizeA[1] = np.array(SizeA[1],dtype=type(1.))
    SizeA[2] = np.array(SizeA[2],dtype=type(1.))
    SizeA[3] = np.array(SizeA[3],dtype=type(1.))
    SigmaA[0] = np.array(SigmaA[0],dtype=type(1.))
    SigmaA[1] = np.array(SigmaA[1],dtype=type(1.))
    SigmaA[2] = np.array(SigmaA[2],dtype=type(1.))
    SigmaA[3] = np.array(SigmaA[3],dtype=type(1.))
    CDA[0] = np.array(CDA[0],dtype=type(1.))
    CDA[1] = np.array(CDA[1],dtype=type(1.))
    LuminA[0] = np.array(LuminA[0],dtype=type(1.))
    LuminA[1] = np.array(LuminA[1],dtype=type(1.))
    SIDS[0] = np.array(SIDS[0],dtype=type(1.))
    SIDS[1] = np.array(SIDS[1],dtype=type(1.))
    MOM0_FLUX[0] = np.array(MOM0_FLUX[0],dtype=type(1.))
    MOM0_FLUX[1] = np.array(MOM0_FLUX[1],dtype=type(1.))
    Distances[0] = np.array(Distances[0],dtype=type(1.))
    Distances[1] = np.array(Distances[1],dtype=type(1.))
    V_rms_err[0] = np.array(V_rms_err[0],dtype=type(1.))
    V_rms_err[1] = np.array(V_rms_err[1],dtype=type(1.))
    
    return np.array(SizeA),np.array(SigmaA),np.array(CDA),np.array(LuminA),np.array(SIDS),np.array(MOM0_FLUX),np.array(Distances),np.array(V_rms_err)

                        
                        
                        
                        
#For the ngc253 data we need another function

def Dendro_Arrays_NGC(Dendrogram,LineData,DataVel,ContData,metadata,ColD = True,beam_size=999,beam_req = 999999,Edge_Cases=True,Trunks=True,max_size=1):
    SizeA,SigmaA,LuminA,CDA,SIDS,MOM0_FLUX,Distances,V_rms_err = [[],[],[],[]],[[],[],[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]]
    print(metadata)
    
    d_copy= Dendrogram
    #catalog = astrodendro.ppv_catalog(d, metadata)
    center = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs')
    center_ra_pix,center_dec_pix = int(metadata['wcsu'][:][:][0].world_to_pixel(center)[0]),int(metadata['wcsu'][:][:][0].world_to_pixel(center)[1])
    sliced= LineData[6]
    CubeShape = np.shape(sliced)
    for t in Dendrogram.all_structures: 

        I = t.indices()
        Cont = True
        if t.is_branch:
                if t.parent==None:
                    if Trunks:
                        Cont=True
                    else:
                        Cont=False
                else:
                    Cont = True
        
        for lmi in range(len(I[0])):
            NansNE=0
            NansSE=0
            NansNW=0
            NansSW=0
            Length = 10
            #I[1][lmi+10]>
            if Edge_Cases==False:
                for lmj in range(Length):
                    #Check four 45 degree prongs from each point and see if they have at least 7 nans in 10 pixels. If that happens its too close to the boundary
                    try:

                        if(sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]<0 ):
                            pass
                        else:
                            NansSE+=1
                        if(sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]<0 ):
                            pass
                        else:
                            NansSW+=1
                        if(sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]<0 ):
                            pass
                        else:
                            NansNW+=1
                        if(sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]<0 ):
                            pass
                        else:
                            NansNE+=1
                    except:
                        #only fails if the I goes close to the boundary of the cube and tries to get a pixel outside the cube
                        Cont = False
                if(NansNE>Length-3 or NansNW>Length-3 or NansSE>Length-3 or NansSW>Length-3):
                    Cont = False

                    break

        if(Cont):
            s = PPVStatistic(t,metadata=metadata)
            s_radius = s.radius
            s_v_rms = s.v_rms
            #print(np.nanmax(I[0]) - np.nanmin(I[0]) )
            #if((float((s_radius*np.pi/180*3.5/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*3.5/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01 and (np.nanmax(I[0]) - np.nanmin(I[0]))*(float((s_radius*np.pi/180*3.5/u.deg)))*10**6 >3*3):
            if((float((s_radius*np.pi/180*3.5/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*3.5/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01 and (np.nanmax(I[0]) - np.nanmin(I[0]))>3):
            #if((float((s_radius*np.pi/180*3.5/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*3.5/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01):
            
            

                nproj_pix=len(set(zip(*tuple(I[i] for i in [1,2]))))
                v_IWM = np.nansum(LineData[I]*(DataVel[I[0]])/u.km*u.s)/np.nansum(LineData[I])
                sig_Sh = np.sqrt(np.nansum(LineData[I]*((DataVel[I[0]])/u.km*u.s-v_IWM)**2)/np.nansum(LineData[I])) 
                
                #The flux from the continuum
                #Convert to Jansky from Jansky per beam:
                if(ColD ==True):
                    Cont_Flux=0

                    proj = tuple(set(zip(*tuple(I[i] for i in [1,2]))))
                    for lmi in range(len(proj)):

                        Cont_Flux+=ContData[proj[lmi]]
                    Cont_Flux=Cont_Flux/(metadata['beam_area_ratioc']*(2*np.sqrt(2*np.log(2))))*u.pix**2*u.beam/u.beam*u.Jy#SHould be input as Jansky /beam and will be converted to Jansky, then to unitless. The beam is changed from FWHM to Gaussian
                    Dust_Column = Flux_to_Mass(Cont_Flux)*Num_per_kg/((s_radius*np.pi/180*3.5/u.deg)**2*(3.086*10**24)**2)/np.pi*(1.989*10**30*u.kg/u.M_sun)/u.kg
                else:
                    Dust_Column=0
                if(str(Dust_Column) == str(np.nan) or str(Dust_Column)==str(np.inf)):
                    Dust_Column=0
                lum = Flux_to_Lum(s.flux)
                s_flux = s.flux

                Index = tuple(I[i] for i in [0,1,2])
                K_Km_s_Flux=np.nansum(LineData[Index]*metadata["velocity_scale"])#Find the total flux from the structures in K km/s, assuming the input data is in K as it should be, 
                
                
                
                
                Distance = np.sqrt((float(s.x_cen/u.pix)-center_ra_pix)**2+(float(s.y_cen/u.pix)- center_dec_pix)**2)*metadata['spatial_scale']*np.pi/180*3.5*10**6/u.deg#pc dist from barycenter
                
                
                V_err= 0#Get_V_rms_err(dend1=d_copy,idx=int(t.idx),struct=t,m=m,NF=1,iterations=5,metadata=metadata)
                
                
                if(t.is_leaf):

                    SizeA[0].append((float((s_radius*np.pi/180*3.5/u.deg)))) #define size as astrodendro
                    SigmaA[0].append((float(s_v_rms/u.km*u.s)))#
                    CDA[0].append(float(Dust_Column))
                    LuminA[0].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[0].append(float(t.idx))
                    MOM0_FLUX[0].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[0].append(float(Distance))
                    V_rms_err[0].append(float(V_err))
                if(t.is_branch	):

                    SizeA[1].append((float((s_radius*np.pi/180*3.5/u.deg)))) #define size as astrodendro
                    SigmaA[1].append((float(s_v_rms/u.km*u.s)))#
                    CDA[1].append(float(Dust_Column))
                    LuminA[1].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[1].append(float(t.idx))
                    MOM0_FLUX[1].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[1].append(float(Distance))
                    V_rms_err[1].append(float(V_err))
                del s
                    
                    
                    
    SizeA[0] = np.array(SizeA[0],dtype=type(1.))
    SizeA[1] = np.array(SizeA[1],dtype=type(1.))
    SizeA[2] = np.array(SizeA[2],dtype=type(1.))
    SizeA[3] = np.array(SizeA[3],dtype=type(1.))
    SigmaA[0] = np.array(SigmaA[0],dtype=type(1.))
    SigmaA[1] = np.array(SigmaA[1],dtype=type(1.))
    SigmaA[2] = np.array(SigmaA[2],dtype=type(1.))
    SigmaA[3] = np.array(SigmaA[3],dtype=type(1.))
    CDA[0] = np.array(CDA[0],dtype=type(1.))
    CDA[1] = np.array(CDA[1],dtype=type(1.))
    LuminA[0] = np.array(LuminA[0],dtype=type(1.))
    LuminA[1] = np.array(LuminA[1],dtype=type(1.))
    SIDS[0] = np.array(SIDS[0],dtype=type(1.))
    SIDS[1] = np.array(SIDS[1],dtype=type(1.))
    MOM0_FLUX[0] = np.array(MOM0_FLUX[0],dtype=type(1.))
    MOM0_FLUX[1] = np.array(MOM0_FLUX[1],dtype=type(1.))
    Distances[0] = np.array(Distances[0],dtype=type(1.))
    Distances[1] = np.array(Distances[1],dtype=type(1.))
    V_rms_err[0] = np.array(V_rms_err[0],dtype=type(1.))
    V_rms_err[1] = np.array(V_rms_err[1],dtype=type(1.))
    
    return np.array(SizeA),np.array(SigmaA),np.array(CDA),np.array(LuminA),np.array(SIDS),np.array(MOM0_FLUX),np.array(Distances),np.array(V_rms_err)





# Cluster finding

Currently unused, but this can find the star forming clusters as detected by Levy 2022

In [ ]:
    
def Read_Clusters(FileName):
    
    sh= len(np.genfromtxt(FileName,usecols=0))
    Data=[]
    for lmi in range(50):
        try:
            Data.append(np.genfromtxt(FileName,usecols=lmi,dtype=type("2d4m")))
            #print(np.genfromtxt(FileName,usecols=lmi,dtype=type("2d4m"),skip_header=1))
        except:
            pass
    return Data
def Find_Clusters_NGC(Data):
    for lmi in range(len(Data)):
        if "ID" in Data[lmi]:
            IDs= Data[lmi][1:9999]
        if "RA" in Data[lmi]: 
            RAs= Data[lmi][1:9999]
        if "Dec" in Data[lmi]:
            Decs= Data[lmi][1:9999]
        if "r_deconv" in Data[lmi]: 
            R_deconv= Data[lmi][1:9999]#pc
        if "glon" in Data[lmi]: 
            glons= Data[lmi][1:9999]#
        if "glat" in Data[lmi]: 
            glats= Data[lmi][1:9999]#
            
    return IDs,RAs,Decs,R_deconv
#Take the cont in Jy and find the HWHM from the structures in the catalog
def Find_Clusters(Data,wcs,Cont_Data,header):
    for lmi in range(len(Data)):
        if "ID" in Data[lmi]:
            IDs= Data[lmi][1:9999]
        if "RA" in Data[lmi]: 
            RAs= Data[lmi][1:9999]
        if "Dec" in Data[lmi]:
            Decs= Data[lmi][1:9999]
        if "r_deconv" in Data[lmi]: 
            R_deconv= Data[lmi][1:9999]#pc
        if "glon" in Data[lmi]: 
            glons= Data[lmi][1:9999]#
        if "glat" in Data[lmi]: 
            glats= Data[lmi][1:9999]#
        if "herschel_column" in Data[lmi]: 
            CD= (Data[lmi][1:9999])#pc
            
        if "flux_integrated" in Data[lmi]: 
            Flux_1p3mm= Data[lmi][1:9999]#pc
    #remove nan 
    for lmii in range(len(CD)):
        try:
            if CD[lmii]=='np.nan':
                CD= np.delete(CD, lmii)
                Flux_1p3mm= np.delete(Flux_1p3mm, lmii)
                IDs= np.delete(IDs, lmii)
                glats= np.delete(glats, lmii)
                glons= np.delete(glons, lmii)
                
        except:
            CD = np.array(CD,dtype=type(1.2**5))#float
            break
    glats_New=[]
    glons_New=[]
    CDs_New=[]
    IDs_New=[]
    Flux_1p3mm_New=[]

    #print(CD,sorted(CD),type(CD),type(CD[0]))
    nth = sorted(CD)[len(CD)-34]#34 most dense leaves
    #print(nth,"A",CD,sorted(CD))
    for lmj in range(len(CD)):
        if CD[lmj]>nth:
            glats_New.append(glats[lmj])
            glons_New.append(glons[lmj])
            CDs_New.append(CD[lmj])
            IDs_New.append(int(IDs[lmj]))
            Flux_1p3mm_New.append(Flux_1p3mm[lmj])
    HWHM_rad = []      
    #print(Flux_1p3mm_New,glats_New,glons_New,CDs_New,IDs_New)
    for lmi in range(len(CDs_New)):
        glat = glats_New[lmi]
        glon = glons_New[lmi]
        Flux = float(Flux_1p3mm_New[lmi])#INtegerated flux in jy
        
        Circle_R = 0
        distance = 8.178*10**-3*u.Mpc
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(l=float(glon)*u.deg, b=float(glat)*u.deg, frame='galactic')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        
        while(True):
            Circle_R += .01
            #pixels=[(p1,p2)]
            pixels=[(p2,p1)]#Goes lat then long for the cont data
            #print(p1,p2)
            #print(np.shape(Cont_Data[p2-50:p2+50]))
            #print(np.shape(Cont_Data[50,p1-50:p1+50]))
            for lmii in range(np.shape(Cont_Data[p2-50:p2+50])[0]):
                for lmjj in range(np.shape(Cont_Data[p2-50+lmii,p1-50:p1+50])[0]):
                    #Find pixels within the circle around the center (excude the center since its there already)
                    #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                    if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                        pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
                        
            
            
            sum_flux=0
            for lmkk in range(len(pixels)):
                sum_flux += (Cont_Data[pixels[lmkk]])
            #print(p1,p2,glat,glon,np.shape(Cont_Data),pixels,Cont_Data[pixels[0]],Flux,sum_flux,Circle_R)
            if sum_flux>Flux/2:
                HWHM_rad.append(Circle_R)#Pc
                break
                
    return HWHM_rad,CDs_New,glons_New,glats_New,IDs_New

#Return masked data around clusters or one pc around clusters
def Mask_Clusters_NGC(HWHM,wcs,header,unmasked_data,ras,decs,One_Pc=False,One_Pc_Size=1,HWHM_Fac=1):
    
    Masked_Data=copy.deepcopy(unmasked_data)
    for lmi in range(len(HWHM)):
        ra = ras[lmi]
        dec = decs[lmi]
                
        Circle_R = HWHM[lmi]*HWHM_Fac
        if(One_Pc):
            
            Circle_R=One_Pc_Size
        distance = 3.5*u.Mpc
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(str(ra),str(dec), frame='icrs')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        


        #pixels=[(p1,p2)]
        pixels=[(p2,p1)]#Goes lat then long for the cont data
        #print(p1,p2)
        #print(np.shape(Cont_Data[p2-50:p2+50]))
        #print(np.shape(Cont_Data[50,p1-50:p1+50]))
        for lmii in range(np.shape(unmasked_data[0,p2-50:p2+50])[0]):
            for lmjj in range(np.shape(unmasked_data[0,p2-50+lmii,p1-50:p1+50])[0]):
                #Find pixels within the circle around the center (excude the center since its there already)
                #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                
                if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                    pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
        
        for lmi in range(len(unmasked_data)):
            
            for lmj in range(len(pixels)):
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
                Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]]=np.nan
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
     
    return Masked_Data
            
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,1,True)
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,2,True)


def Mask_Clusters_CMZ(HWHM,wcs,header,unmasked_data,glons,glats,One_Pc=False,One_Pc_Size=1,HWHM_Fac=1):
    
    Masked_Data=copy.deepcopy(unmasked_data)
    for lmi in range(len(HWHM)):
        glon = glons[lmi]
        glat = glats[lmi]
                
        Circle_R = HWHM[lmi]*HWHM_Fac
        if(One_Pc):
            
            Circle_R=One_Pc_Size
        distance = dist_cmz
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(float(glon)*u.deg,float(glat)*u.deg, frame='galactic')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        


        #pixels=[(p1,p2)]
        pixels=[(p2,p1)]#Goes lat then long for the cont data
        #print(p1,p2)
        #print(np.shape(Cont_Data[p2-50:p2+50]))
        #print(np.shape(Cont_Data[50,p1-50:p1+50]))
        for lmii in range(np.shape(unmasked_data[0,p2-50:p2+50])[0]):
            for lmjj in range(np.shape(unmasked_data[0,p2-50+lmii,p1-50:p1+50])[0]):
                #Find pixels within the circle around the center (excude the center since its there already)
                #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                
                if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                    pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
        
        for lmi in range(len(unmasked_data)):
            
            for lmj in range(len(pixels)):
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
                Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]]=np.nan
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
     
    return Masked_Data
            
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,1,True)
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,2,True)



# Useful functions

In [ ]:

#Make an image from the SC data

#Data=the image data to plot
#vmin,vmax=the min/max scale for the graph's color bar
#Name2, another descriptor

def Make_Plot(Pointing_Information,Name2,Data,vmin,vmax,rows=1,columns=1,index=1,show=False):
    
    Name=Pointing_Information['target']
    WCS=Pointing_Information['wcsu']
    Glon = str(WCS).find("GLON")!=-1 #If the wcs is in Glon or not
    
    # Build a plot
    ax = pylab.subplot(rows,columns,index,projection=WCS) 
    RA = ax.coords[0]                                                                  # 
    Dec = ax.coords[1]
    im = pylab.imshow(Data,vmin=vmin,vmax=vmax,cmap='rainbow')
    RA.set_ticks(size=-3)                                                                                      
    Dec.set_ticks(size=-3) 
    RA.set_ticklabel(exclude_overlapping=True) 
    Dec.set_ticklabel(exclude_overlapping=True)                                                                                     
    
    if(Glon==False):
        pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
        pylab.ylabel('Declination',fontsize=20,labelpad=1)
    else:
        pylab.xlabel('Galatic longitude',fontsize=20,labelpad=1)                               
        pylab.ylabel('Galatic latitude',fontsize=20,labelpad=1)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 15)    
    cb=pylab.colorbar(im,fraction=0.1,pad=0.0)                                     
    cb.set_label(label=Name,fontsize=10,rotation=270,labelpad=20) 
    cb.ax.tick_params(which = 'major', labelsize = 10)   
    pylab.annotate(text=Name2,fontsize=10,xy=(0.02,1.05),xycoords="axes fraction")  
    
    if(show==True):
        pylab.show()
        
        
#Put this up here for the column density map
# Make sure the input flux is in Jy
# And make sure the input distance is in Mpc

# This is a mass conversion factor for CO 3-2 to H2 calibrated using the 850 Ghz dust continuum
# It will not be used widly, but if it was I would need a relation factor for each molecular based 
# on their relative abundance, and their line transition ratio.
# This has also been divided by two to account for the higher metallicity of NGC253 as compared to the CMZ
# as Krieger did in 2020

a_850 = 6.7*10**19*u.erg/u.s/u.Hz/u.M_sun #6.7+-1.7, from Bolatto 2013a


def Flux_to_Mass(flux, dist, Lum_per_mass_factor=a_850):
    
    #Here is the manual process to convert to ergs
    #J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy #Jansky to flux in erg/(s cm^2 Hz)
    #flux_erg = flux*J_to_e
    flux_erg = flux.to(u.erg)
    #here is the conversion factor for Mpc to cm
    #Mpc_to_cm = 3.086*10**24 * u.cm/u.Mpc
    dist_cm=dist.to(u.cm)
    
    # Now use the distance and flux to calculate the luminosity, then use that and the conversion factor to find the mass
    
    # L = 4pi*r2 * Flux
    
    L = 4*np.pi*(dist_cm)**2*flux_erg #Megaparsec is converted to cm
    
    
    
    
    
    M_mol = L/Lum_per_mass_factor #Just in Solar mass*1.989*10**30*u.kg/u.M_sun #Determines mass of the cont for 850 in kg
    
    return M_mol

        
# This assumes the input flux will be in Jansky, which the astrodendro library defaults to.
# And make sure the input distance is in Mpc

def Flux_to_Lum(flux, dist):
    
    #Here is the manual process to convert to ergs
    #J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy #Jansky to flux in erg/(s cm^2 Hz)
    #flux_erg = flux*J_to_e
    flux_erg = flux.to(u.erg)
    #here is the conversion factor for Mpc to cm
    #Mpc_to_cm = 3.086*10**24 * u.cm/u.Mpc
    dist_cm=dist.to(u.cm)
    
    # Now use the distance and flux to calculate the luminosity, then use that and the conversion factor to find the mass
    
    # L = 4pi*r2 * Flux
    
    L = 4*np.pi*(dist_cm)**2*flux_erg #Return the luminosity in Erg

    
    return L


def Get_V_rms_err(dend1,struct,idx,m,NF,iterations,metadata):
    
    
    vs=[]
    np.random.seed((99)**2*123)
    for llll in range(iterations):
        
        #print(llll)
        s = dend1.__getitem__(idx)
        #s = struct#copy.deepcopy(struct)
        #s2 = struct#copy.deepcopy(struct)
        npixels = np.product(np.shape(s.values()))
        #print(np.shape(s.values()),s.values())
        
        additional_noise = np.random.normal(0., m*NF, npixels)
        additional_noise = np.reshape(additional_noise, np.shape(s.values()))
        #add or subract noise to the values and calculate the v rms, them find the std of that array and
        # call that the uncertainty in v rms for a structure
        dat1P = dend1.data[s.indices()]
        dend1.data[s.indices()]+= additional_noise
        s = dend1.__getitem__(idx)
        vs.append(float(PPVStatistic(s,metadata=metadata).v_rms/u.km*u.s))
        dend1.data[s.indices()]= dat1P#reset the dend data
        
        dend1.data[s.indices()]-= additional_noise
        #s._values+=additional_noise
        #print(s.values(),s._values)
        
        #s2._values-=additional_noise
        s = dend1.__getitem__(idx)
        #print(dat1P[0],s._values[0],"kaasl")
        vs.append(float(PPVStatistic(s,metadata=metadata).v_rms/u.km*u.s))
        
        del s
        #del s2
        
    v_rms_std = np.nanstd(vs)
    #print(v_rms_std)
    return v_rms_std

# Return a cropped cube for some ra and dec, also crops the velocity axis if needed (0 for no crop)
# cube= the spetral cube you wish to crop
# WCS = that cube's world coordinate system (.WCS)
# Np1, Np2 = the first and final pixel that don't have nan values (the left and right bounds of the cube)
# BadVel = put a number based on the amound of velocity channels that are just noise, or leave at zero if you want to keep the noise
# D2 = put True if the cube is 2D, otherwise, put False

def Crop(cube,WCS,Np1,Np2,BadVel,D2):
    NraDP1 = [int(WCS.world_to_pixel(Np1)[0]),int(WCS.world_to_pixel(Np1)[1])]
    NraDP2 = [int(WCS.world_to_pixel(Np2)[0]),int(WCS.world_to_pixel(Np2)[1])]
    if(D2==False):
        return cube[BadVel:np.shape(cube)[0]-BadVel,NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]
    if(D2==True):
        return cube[NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]

    
    

def Crop_Nans(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
                
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey



# More useful functions

In [ ]:



def gaussian_beam(f, beam_gauss_width):
    '''
    Fourier transform of a Gaussian beam. NOT the power spectrum (multiply exp
    argument by 2 for power spectrum).
    Parameters
    ----------
    f : np.ndarray
        Frequencies to evaluate beam at.
    beam_gauss_width : float
        Beam size. Should be the Gaussian rms, not FWHM.
    '''
    return np.exp(-f**2 * np.pi**2 * 2 * beam_gauss_width**2)

def gauss_correlated_noise_2D(shape, sigma, beam_gauss_width,
                              randomseed=327485749):
    
    '''
    Generate correlated Gaussian noise with sigma, smoothed by a
    Gaussian kernel.
    '''

    # Making a real signal. Only need real part of FFT
    freqs_yy, freqs_xx = np.meshgrid(np.fft.fftfreq(shape[0]),
                                     np.fft.rfftfreq(shape[1]), indexing="ij")

    freqs = np.sqrt(freqs_yy**2 + freqs_xx**2)
    # freqs[freqs == 0.] = np.NaN
    # freqs[freqs == 0.] = 1.

    imsize = shape[0]

    Np1 = (imsize - 1) // 2 if imsize % 2 != 0 else imsize // 2
    
    with NumpyRNGContext(randomseed):

        angles = np.random.uniform(0, 2 * np.pi,
                                   size=freqs.shape)

    noise = np.cos(angles) + 1j * np.sin(angles)

    if imsize % 2 == 0:
        noise[1:Np1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1, -1] = np.conj(noise[imsize:Np1:-1, -1])
        noise[Np1, 0] = noise[Np1, 0].real + 1j * 0.0
        noise[Np1, -1] = noise[Np1, -1].real + 1j * 0.0

    else:
        noise[1:Np1 + 1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1 + 1, -1] = np.conj(noise[imsize:Np1:-1, -1])

    # Zero freq components must have no imaginary part to be own conjugate
    noise[0, -1] = noise[0, -1].real + 1j * 0.0
    noise[0, 0] = noise[0, 0].real + 1j * 0.0

    corr_field = np.fft.irfft2(noise *
                               gaussian_beam(freqs, beam_gauss_width))

    norm = (np.sqrt(np.sum(corr_field**2)) / np.sqrt(corr_field.size)) / sigma

    corr_field /= norm
    
    return corr_field








  

        
        


# Noise matching

this is an unused function that is from Krieger 2020, where he adds additional noise to the CMZ image to match the noise present in the NGC253
image. I do not do this because it creates a ton of false structures that dont exist and I dont want those in the CMZ data.

I require at least 5 times the noise from a noise-channel before I allow the pixels to be considered real data.
I calculate the noise for each cube after doing all the data reduction, which is expected to amplify the noise.

In [ ]:
        
def Noise_matching(Input_Cube_Match,Input_Cube_Noisy,Cube_Name_Save,Force_region=False,FVX=0,FVY=0,Force_Noise=False,FNV1=0,FNV2=0):
    
    
    

    #Find noise for ngc253
    

    Qp = Input_Cube_Noisy.with_spectral_unit(u.km/u.s,velocity_convention="radio") 
    Qp.allow_huge_operations=True

    Qp = Qp.to(u.K)#Jy to Kelvin

    datn = Qp.hdu.data
    del Qp


    Non_nan=((datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]>0)  | (datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]<0 ))

    NGCCO32_Noise = (np.nanstd(datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)],where= Non_nan)) #Noise K
    print(np.shape(datn))
    del datn





    Qp = Input_Cube_Match
    #match noise
    Qp.allow_huge_operations=True
    Q = Qp.to(u.K)#Jy to Kelvin
    del Qp
    datn=Q.hdu.data
    
    if gal =="GC":
        Non_nan=((datn[0,:,0:int(np.shape(datn)[2]/2)]>0)  | (datn[0,:,0:int(np.shape(datn)[2]/2)]<0 ))

        m = (np.nanstd(datn[0,:,0:int(np.shape(datn)[2]/2)],where= Non_nan)) #Noise K
    else:
        Non_nan=((datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]>0)  | (datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]<0 ))

        m = (np.nanstd(datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)],where= Non_nan)) #Noise K

    if (Force_Noise):
        m = FNV1 #Force a noise Values .037? .115?
        NGCCO32_Noise=FNV2
    print(m,"Noise (K) matched to ",NGCCO32_Noise)





    npixels = np.product(Q.hdu.data.shape)

    target_noise = float(NGCCO32_Noise)
    actual_noise = m
    additional_sigma = np.sqrt(np.abs(target_noise**2 - actual_noise**2))

    additional_noise = np.random.normal(0., additional_sigma, npixels)
    additional_noise = np.reshape(additional_noise, Q.hdu.data.shape)


    fwhm_factor = np.sqrt(8*np.log(2))
    add_noise = np.zeros(np.shape(datn))
    for lmi in range(len(datn)):
        new_seed = np.random.randint(1e9)
        additional_noise = gauss_correlated_noise_2D(shape=(Q.hdu.data[6].shape[0],Q.hdu.data[6].shape[1]), sigma=additional_sigma, beam_gauss_width=5/fwhm_factor,randomseed=new_seed)
        pp=np.where(additional_noise!=np.nan)
        add_noise[lmi][pp]=additional_noise[pp]

    new_data = datn+add_noise
    QCopy = Q.hdu
    QCopy.data = new_data
    Q = SpectralCube.read(QCopy)
    del QCopy

    

    Q.write(Cube_Name_Save,overwrite=True)
    del Q
    print("Wrote to",Cube_Name_Save)
    
    
 